In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 表示設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.unicode.east_asian_width', True)

# データ読み込みと前処理
df = pd.read_csv("データ.csv")
df = df.fillna(0)

# シーン列をバイナリ列に変換
df["普段使い"] = df["シーン"].apply(lambda x: 1 if "普段使い" in x else 0)
df["ビジネス"] = df["シーン"].apply(lambda x: 1 if "ビジネス" in x else 0)
df["切り替え"] = df["シーン"].apply(lambda x: 1 if "切り替え" in x else 0)
df["特別な時"] = df["シーン"].apply(lambda x: 1 if "特別な時" in x else 0)
df = df.drop(columns=["シーン"])

# 性別列をバイナリ列に変換
df["男性"] = df["性別"].apply(lambda x: 1 if x in ["男性", "ユニセックス"] else 0)
df["女性"] = df["性別"].apply(lambda x: 1 if x in ["女性", "ユニセックス"] else 0)
df = df.drop(columns=["性別"])

df["濃度"] = df["濃度"].map({"EDT": 5, "EDP": 10, "EDC": 1})

# 標準化
columns_to_scale = df.loc[:, '価格':].columns
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# K-Means クラスタリング
kmeans = KMeans(n_clusters=6, random_state=42)
df['Cluster'] = kmeans.fit_predict(df.loc[:, '価格':])

# クラスタリング結果の確認
cluster_summary = df.groupby('Cluster').mean()
print(cluster_summary)

# --- リコメンドシステム ---
def get_user_input():
    """
    ユーザーからの入力を取得する関数
    """
    print("\nリコメンドシステムへようこそ！以下の項目を入力してください。\n")
    
    # 価格を入力
    price = float(input("価格を入力してください（例: 10000）: "))
    
    # 性別を入力
    gender = input("性別を入力してください（男性/女性/ユニセックス）: ").strip()
    male = 1 if gender in ["男性", "ユニセックス"] else 0
    female = 1 if gender in ["女性", "ユニセックス"] else 0
    
    # 使用シーンを入力
    scene = input("使用シーンをカンマ区切りで入力してください（例: 普段使い, ビジネス, 特別な時）: ").lower().split(',')
    scene_flags = {
        "普段使い": 1 if "普段使い" in scene else 0,
        "ビジネス": 1 if "ビジネス" in scene else 0,
        "切り替え": 1 if "切り替え" in scene else 0,
        "特別な時": 1 if "特別な時" in scene else 0,
    }
    
    # 好みの香りを入力
    fragrance = input("好みの香りをカンマ区切りで入力してください（例: フルーティ, ウッディ, ムスク）: ").lower().split(',')
    fragrance_mapping = {
        "フルーティ": 1, "ホワイトフローラル": 2, "フローラル": 3, "ウッディ": 4,
        "シトラス": 5, "ムスク": 6, "グリーンフローラル": 7, "アロマティック": 8,
        "アンバー": 9, "アクア": 10, "アニマリック": 11, "パウダリー": 12,
        "グルマン": 13, "グリーン": 14, "スパイシー": 15, "オリエンタル": 16,
        "ハーバル": 17
    }
    fragrance_flags = {key: (1 if key in [f.strip() for f in fragrance] else 0) for key in fragrance_mapping.keys()}
    
    # 濃度を入力（1～10の範囲）
    while True:
        concentration = float(input("濃度を1～10の間で入力してください（例: 5）: "))
        if 1 <= concentration <= 10:
            break
        else:
            print("濃度は1～10の範囲で入力してください。")
    
    # 入力データをリスト化
    user_input = [price, male, female, *scene_flags.values(), *fragrance_flags.values(), concentration]
    return user_input

def recommend(input_data, df, feature_columns):
    """
    入力データに基づき、リコメンド結果を返す関数
    """
    # クラスタ予測
    cluster_label = kmeans.predict([input_data])[0]
    print(f"\n入力データはクラスタ {cluster_label} に属しています。\n")
    
    # 同じクラスタのデータを抽出
    similar_items = df[df['Cluster'] == cluster_label]
    
    # 類似度計算
    similarities = cosine_similarity([input_data], similar_items[feature_columns])
    similar_items = similar_items.copy()
    similar_items['Similarity'] = similarities[0]
    
    # 類似度の降順でソートして上位を返す
    recommendations = similar_items.sort_values(by='Similarity', ascending=False)
    return recommendations

# --- 実行部分 ---
user_input = get_user_input()
scaled_user_input = scaler.transform([user_input])[0]  # 標準化

# リコメンド実行
recommendations = recommend(
    input_data=scaled_user_input,
    df=df,
    feature_columns=df.columns[:-1]  # 最後の列 'Cluster' を除外
)

# 結果の表示
print("\nリコメンド結果:")
print(recommendations.head())


             価格  フルーティ  ホワイトフローラル  フローラル  ウッディ  シトラス    ムスク  グリーンフローラル  アロマティック  パウダリー  グルマン  グリーン  スパイシー  オリエンタル  ハーバル  アンバー    アクア  アニマリック      濃度        春        夏        秋        冬  普段使い  ビジネス  切り替え  特別な時      男性      女性
Cluster                                                                                                                                                                                                                                                                                                                                            
0        0.656579    0.431682            0.325582   -0.125000 -0.599903 -0.597868 -0.905539           -0.461880       -0.408248   -0.631995 -0.275046 -0.430394    0.716478      4.663690 -0.214423 -0.288675 -0.214423     -0.105409  0.185062 -0.716478 -0.339813  0.213497  0.266501 -0.229535 -0.547723  0.504675  0.266501 -0.578638  0.331295
1        0.026388   -0.441275            0.839673   -0.522222 -0.684540 -0.687795  0.032393    

ValueError: operands could not be broadcast together with shapes (1,25) (29,) (1,25) 

In [ ]:
pd.set_option('display.max_rows', None)  # 全行を表示
pd.set_option('display.max_columns', None)  # 全列を表示
pd.set_option('display.expand_frame_repr', False)  # 列幅で改行させない
pd.set_option('display.unicode.east_asian_width', True)  # 日本語を整列させる

In [ ]:
df = pd.read_csv("データ.csv")
df = df.fillna(0)

df

In [ ]:
# シーン列をバイナリ列に変換
df["普段使い"] = df["シーン"].apply(lambda x: 1 if "普段使い" in x else 0)
df["ビジネス"] = df["シーン"].apply(lambda x: 1 if "ビジネス" in x else 0)
df["切り替え"] = df["シーン"].apply(lambda x: 1 if "切り替え" in x else 0)
df["特別な時"] = df["シーン"].apply(lambda x: 1 if "特別な時" in x else 0)
df = df.drop(columns=["シーン"])
df

In [ ]:
df["男性"] = df["性別"].apply(lambda x: 1 if x in ["男性", "ユニセックス"] else 0)
df["女性"] = df["性別"].apply(lambda x: 1 if x in ["女性", "ユニセックス"] else 0)
df = df.drop(columns=["性別"])
df

# 濃度を数値に変換
df["濃度"] = df["濃度"].map({"EDT": 5, "EDP": 10, "EDC": 1})
df

In [ ]:
df["濃度"] = df["濃度"].map({"EDT": 5, "EDP": 10, "EDC": 1})
df

In [ ]:
# 標準化対象のカラムを指定
columns_to_scale = df.loc[:, '価格':].columns
# StandardScalerのインスタンスを作成
scaler = StandardScaler()
# 対象列を標準化してデータフレームに戻す
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
print(df.isnull().any().any())
df

In [ ]:
inertia = []
K = range(1, 11)  # クラスタ数を1〜10で試す

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df.loc[:, '価格':'女性'])
    inertia.append(kmeans.inertia_)

# エルボー法のプロット
plt.figure(figsize=(8, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
for k in range(2, 11):  # クラスタ数を2～10で試す
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(df.loc[:, '価格':'女性'])
    score = silhouette_score(df.loc[:, '価格':'女性'], labels)
    silhouette_scores.append(score)

# 結果をプロット
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.title('Silhouette Analysis')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42)
df['Cluster'] = kmeans.fit_predict(df.loc[:, '価格':])
df

In [ ]:
cluster_summary = df.groupby('Cluster').mean()
cluster_summary

In [ ]:
def recommend(input_data, df, feature_columns):
    """
    入力データに基づき、リコメンド結果を返す関数
    """
    # クラスタ予測
    cluster_label = kmeans.predict([input_data])[0]
    print(f"\n入力データはクラスタ {cluster_label} に属しています。\n")
    
    # 同じクラスタのデータを抽出
    similar_items = df[df['Cluster'] == cluster_label]
    
    # 類似度計算
    similarities = cosine_similarity([input_data], similar_items[feature_columns])
    similar_items = similar_items.copy()
    similar_items['Similarity'] = similarities[0]
    
    # 類似度の降順でソートして上位を返す
    recommendations = similar_items.sort_values(by='Similarity', ascending=False)
    return recommendations

In [ ]:
# --- リコメンドシステム ---
def get_user_input():
    """
    ユーザーからの入力を取得する関数
    """
    print("\nリコメンドシステムへようこそ！以下の項目を入力してください。\n")
    
    # 価格を入力
    price = float(input("価格を入力してください（例: 10000）: "))
    
    # 性別を入力
    gender = input("性別を入力してください（男性/女性/ユニセックス）: ").strip()
    male = 1 if gender in ["男性", "ユニセックス"] else 0
    female = 1 if gender in ["女性", "ユニセックス"] else 0
    
    # 使用シーンを入力
    scene = input("使用シーンをカンマ区切りで入力してください（例: 普段使い, ビジネス, 特別な時）: ").lower().split(',')
    scene_flags = {
        "普段使い": 1 if "普段使い" in scene else 0,
        "ビジネス": 1 if "ビジネス" in scene else 0,
        "切り替え": 1 if "切り替え" in scene else 0,
        "特別な時": 1 if "特別な時" in scene else 0,
    }
    
    # 好みの香りを入力
    fragrance = input("好みの香りをカンマ区切りで入力してください（例: フルーティ, ウッディ, ムスク）: ").lower().split(',')
    fragrance_mapping = {
        "フルーティ": 1, "ホワイトフローラル": 2, "フローラル": 3, "ウッディ": 4,
        "シトラス": 5, "ムスク": 6, "グリーンフローラル": 7, "アロマティック": 8,
        "アンバー": 9, "アクア": 10, "アニマリック": 11, "パウダリー": 12,
        "グルマン": 13, "グリーン": 14, "スパイシー": 15, "オリエンタル": 16,
        "ハーバル": 17
    }
    fragrance_flags = {key: (1 if key in [f.strip() for f in fragrance] else 0) for key in fragrance_mapping.keys()}
    
    # 濃度を入力（1～10の範囲）
    while True:
        concentration = float(input("濃度を1～10の間で入力してください（例: 5）: "))
        if 1 <= concentration <= 10:
            break
        else:
            print("濃度は1～10の範囲で入力してください。")
    
    # 入力データをリスト化
    user_input = [price, male, female, *scene_flags.values(), *fragrance_flags.values(), concentration]
    return user_input

In [ ]:
# --- 実行部分 ---
user_input = get_user_input()
scaled_user_input = scaler.transform([user_input])[0]  # 標準化

# リコメンド実行
recommendations = recommend(
    input_data=scaled_user_input,
    df=df,
    feature_columns=df.columns[:-1]  # 最後の列 'Cluster' を除外
)

# 結果の表示
print("\nリコメンド結果:")
print(recommendations.head())